### lesson-4

## Домашнее задание

Перенесите метрики в модуль metrics.py (убедится что они там)

Перенесите функцию prefilter_items в модуль utils.py

Создайте модуль recommenders.py. Напищите код для класса ниже (задание обсуждали на вебинаре, для первой функции 
практически сделали) и положите его в recommenders.py

Проверьте, что все модули корректно импортируются


### 1. Перенесите метрики в модуль metrics.py

In [3]:
%%writefile "metrics.py"

import numpy as np


def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate
    
    
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate
    
    
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision
    

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision
    

def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = prices_recommended[flags].sum() / prices_recommended.sum()
    
    return precision
    

def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall
    

def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall
    

def money_recall_at_k(recommended_list, bought_list, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = prices_bought[flags].sum() / prices_bought.sum()
    
    return recall
    

def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list,bought_list,k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant
    

def map_k(recommended_lists, bought_lists, k=5):
    
    sum_ = sum([ap_k(recommended_lists[i], bought_lists[i], k=k) for i in range(len(bought_lists))])
    
    return sum_/len(bought_lists)
    

def dcg_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(recommended_list, bought_list).astype(int)
    
    sum_ = sum([flags[i]/np.log2(i+1) if i+1 > 2 else flags[i]/(i+1) for i in range(k)])
    
    return sum_/k
    

def ndcg_at_k(recommended_list, bought_list, k=5):
    
    dcg = dcg_at_k(recommended_list, bought_list, k=k)
    ideal_dcg = sum([1/np.log2(i+1) if i+1 > 2 else 1/(i+1) for i in range(k)])/k
    
    return dcg/ideal_dcg
    

def two_cycles(first_list, second_list):
    for i in first_list:
        for j in second_list:
            yield i, j
    

def reciprocal_rank(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    rank = None
    
    for i, bought in two_cycles(range(len(recommended_list)), bought_list):
        if recommended_list[i] == bought:
            rank = i+1
            break
    
    if rank is None:
        return 0
    
    return 1/rank
    

def mrr_k(recommended_lists, bought_lists, k=5):
    
    sum_ = sum([reciprocal_rank(recommended_lists[i], bought_lists[i], k=k) for i in range(len(bought_lists))])
    
    return sum_/len(bought_lists)

Overwriting metrics.py


### 2. Перенесите функцию prefilter_items в модуль utils.py

In [4]:
%%writefile "utils.py"

import numpy as np


class Prefilter_Items():
    
    def __init__(self, drop_popular=False, drop_not_popular=False, drop_old=False, drop_low_price=False, 
                 drop_high_price=False, drop_not_top_N=None, item_features=None, drop_wrong_department=None):
        
        self.drop_popular = drop_popular
        self.drop_not_popular = drop_not_popular
        self.drop_old = drop_old
        self.drop_low_price = drop_low_price
        self.drop_high_price = drop_high_price
        
        self.drop_not_top_N = drop_not_top_N
        self.item_features = item_features
        self.drop_wrong_department = drop_wrong_department
        
        self.popular = None
        self.not_popular = None
        self.old = None
        self.low_price = None
        self.high_price = None
        self.not_top_N = None
        self.wrong_department = None
    
    def fit(self, data_train):
        
        if self.drop_popular or self.drop_not_popular or (self.drop_not_top_N is not None):
            popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index()
            popularity['user_id'] /= data_train['user_id'].nunique()
            popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
        
        if self.drop_popular:
            self.popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
        
        if self.drop_not_popular:
            self.not_popular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
        
        if self.drop_old:
            self.old = data_train[data_train['week_no'] <= data_train['week_no'].max() - 52].item_id.tolist()
        
        if self.drop_low_price or self.drop_high_price:
            price = data_train[['item_id', 'sales_value']]
            price['sales_value'] /= (np.maximum(data_train['quantity'], 1))
            price = data_train.groupby('item_id')['sales_value'].last().reset_index()
        
        if self.drop_low_price:
            self.low_price = price[price['sales_value'] < 1].item_id.tolist()
        
        if self.drop_high_price:
            self.high_price = price[price['sales_value'] > 100].item_id.tolist()
            
        if self.drop_not_top_N is not None:
            popularity.sort_values('share_unique_users', ascending=True, inplace=True)
            self.not_top_N = popularity.item_id.head(-self.drop_not_top_N).tolist()
        
        if self.drop_wrong_department is not None:
            self.wrong_department = self.item_features.loc[self.item_features['department'].isin(self.drop_wrong_department)]
            self.wrong_department = self.wrong_department.item_id.tolist()
            
        
    def transform(self, data):
        
        # Уберем самые популярные товары (их и так купят)
        if self.drop_popular:
            data = data[~data['item_id'].isin(self.popular)]
        
        # Уберем самые НЕ популярные товары (их и так НЕ купят)
        if self.drop_not_popular:
            data = data[~data['item_id'].isin(self.not_popular)]
        
        # Уберем товары, которые не продавались за последние 12 месяцев
        if self.drop_old:
            data = data[~data['item_id'].isin(self.old)]
        
        # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
        if self.low_price:
            data = data[~data['item_id'].isin(self.low_price)]
        
        # Уберем слишком дорогие товары
        if self.high_price:
            data = data[~data['item_id'].isin(self.high_price)]
            
        # Уберем товары, которые не вошли в топ N
        if self.drop_not_top_N is not None:
            data = data[~data['item_id'].isin(self.not_top_N)]
            
        # Уберем не интересные для рекоммендаций категории (department)
        if self.drop_wrong_department is not None:
            data = data[~data['item_id'].isin(self.wrong_department)]
        
        return data

Writing utils.py


### 3. Создайте модуль recommenders.py. Напищите код для класса ниже (задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py

In [5]:
%%writefile "recommenders.py"

import numpy as np
import pandas as pd

from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix


class MainRecommender:
    
    def __init__(self, data, weighting=True):
        
        self.popularity = self.get_popularity(data)
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        
    @staticmethod
    def get_popularity(data):
        
        popularity = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        popularity.sort_values('quantity', ascending=False, inplace=True)
        
        return popularity
     
    @staticmethod
    def prepare_matrix(data):
        
        data.columns = [col.lower() for col in data.columns]
        
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )
        
        user_item_matrix = user_item_matrix.astype(float)
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model
    
    def get_als_bm25_recommendations(self, user, N=5):
        res = [self.id_to_itemid[rec[0]] for rec in self.model.recommend(userid=self.userid_to_id[user], # userid - id от 0 до N
                           user_items=csr_matrix(self.user_item_matrix).tocsr(),   # на вход user-item matrix
                           N=N, # кол-во рекомендаций 
                           filter_already_liked_items=False, 
                           recalculate_user=False)]
        return res

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        top_N = self.popularity.loc[self.popularity['user_id']==user].head(N)
        
        recommendation_similar_items = set()
        i = 2
        
        while len(recommendation_similar_items) < N:
            for item in top_N['item_id']:
                recommendation_similar_items.add(self.id_to_itemid[self.model.similar_items(self.itemid_to_id[item], 
                                                                                            N=i)[i-1][0]])
                if len(recommendation_similar_items) == N:
                    break
            i+=1
            
        return list(recommendation_similar_items)
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)[1:]
        
        recommendation_similar_users = set()
        i = 1
        
        while len(recommendation_similar_users) < N:
            for user in similar_users:
                recommendation_similar_users.add(self.id_to_itemid[self.own_recommender.recommend(
                                                                        userid=user[0], 
                                                                        user_items=csr_matrix(self.user_item_matrix).tocsr(), 
                                                                        N=i,
                                                                        filter_already_liked_items=False,
                                                                        recalculate_user=True)[i-1][0]])
                if len(recommendation_similar_users) == N:
                    break
            i+=1
            
        return list(recommendation_similar_users)
    
    def recs_top_n(self, N=5):
        res = self.popularity.groupby('item_id')['quantity'].count().reset_index()
        res.sort_values('quantity', ascending=False, inplace=True)
        res = res.item_id.head(N).tolist()
        
        return res

Writing recommenders.py


### 4. Проверьте, что все модули корректно импортируются

In [6]:
import pandas as pd
import numpy as np

from metrics import precision_at_k, recall_at_k
from utils import Prefilter_Items
from recommenders import MainRecommender

In [7]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [8]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [9]:
n_items_train_before = data_train['item_id'].nunique()
n_users_train_before = data_train['user_id'].nunique()

prefilter_items = Prefilter_Items(drop_not_top_N=5000)
prefilter_items.fit(data_train)
data_train = prefilter_items.transform(data_train)

n_items_train_after = data_train['item_id'].nunique()
n_users_train_after = data_train['user_id'].nunique()

print('Decreased # items on train from {} to {}'.format(n_items_train_before, n_items_train_after))
print('Decreased # users on train from {} to {}'.format(n_users_train_before, n_users_train_after))

Decreased # items on train from 86865 to 5000
Decreased # users on train from 2499 to 2479


In [10]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result = result.loc[~result['user_id'].isin(set(data_test['user_id']) - set(data_train['user_id']))]
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [11]:
%%time
mainrecommender = MainRecommender(data_train)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

CPU times: user 23.5 s, sys: 413 ms, total: 23.9 s
Wall time: 5.17 s


In [12]:
%%time
result['als_bm25'] = result['user_id'].apply(lambda x: mainrecommender.get_als_bm25_recommendations(x, N=5))

res_als_bm25 = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
print("Precision:", res_als_bm25)

Precision: 0.15454992621741098
CPU times: user 1min 18s, sys: 1.58 s, total: 1min 20s
Wall time: 10.1 s


In [13]:
%%time
result['similar_items'] = result['user_id'].apply(lambda x: mainrecommender.get_similar_items_recommendation(x, N=5))

res_similar_items = result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean()
print("Precision:", res_similar_items)

Precision: 0.08981800295130322
CPU times: user 15.2 s, sys: 164 ms, total: 15.4 s
Wall time: 1.93 s


In [14]:
%%time
result['similar_users'] = result['user_id'].apply(lambda x: mainrecommender.get_similar_users_recommendation(x, N=5))

res_similar_users = result.apply(lambda row: precision_at_k(row['similar_users'], row['actual']), axis=1).mean()
print("Precision:", res_similar_users)

Precision: 0.01505164781111654
CPU times: user 6min 40s, sys: 8.61 s, total: 6min 49s
Wall time: 51.2 s


In [15]:
%%time
result['baseline_top_n'] = [mainrecommender.recs_top_n(N=5)] * result.shape[0]

res_baseline_top_n = result.apply(lambda row: precision_at_k(row['baseline_top_n'], row['actual']), axis=1).mean()
print("Precision:", res_baseline_top_n)

Precision: 0.19360550909984992
CPU times: user 694 ms, sys: 20.4 ms, total: 714 ms
Wall time: 89.7 ms


In [16]:
result.head(5)

,user_id,actual,als_bm25,similar_items,similar_users,baseline_top_n
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[923670, 1028166, 997796, 1094924, 12352201]","[981760, 845262, 1127825, 1098066, 1033403]","[1058021, 6533681, 1088147, 969494, 907099]","[1082185, 981760, 995242, 1029743, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1110244, 885697, 910032, 951590, 5569230]","[1075979, 1076875, 823704, 1092026, 960318]","[1048483, 826695, 6704207, 9245462, 1027642]","[1082185, 981760, 995242, 1029743, 840361]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 878996, 965267, 1024306, 834484]","[981760, 1004906, 1041259, 948650, 845208]","[1015296, 819840, 962274, 1084036, 8011285]","[1082185, 981760, 995242, 1029743, 840361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[849843, 1082185, 6534178, 1124674, 856335]","[981760, 1094955, 1015247, 871440, 849843]","[891752, 5566472, 949836, 1094190, 964221]","[1082185, 981760, 995242, 1029743, 840361]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[840361, 1082185, 1068719, 981760, 1029743]","[981760, 1079067, 1019744, 932006, 1110843]","[1094924, 827856, 9837076, 1100379, 1044188]","[1082185, 981760, 995242, 1029743, 840361]"


In [17]:
united_table = pd.DataFrame({'\\': ['Results'], 
                             'als_bm25': [res_als_bm25], 
                             'similar_items': [res_similar_items], 
                             'similar_users': [res_similar_users],
                             'baseline_top_n': [res_baseline_top_n]})
united_table

,\,als_bm25,similar_items,similar_users,baseline_top_n
0,Results,0.15455,0.089818,0.015052,0.193606


### Вывод:

На данный момент наш baseline, когда мы всем рекомендуем одинаковые самые популярные товары, показывает самый лучший результат, значит нам нужно дорабатывать наши модели и подбирать лучшие гиперпараметры.